In [ ]:
# imports
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, \
                            roc_auc_score, precision_score, f1_score, recall_score, \
                            mean_squared_error, mean_absolute_error, max_error
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import sys

In [ ]:
# change flag to run local or in colab
colab_env = False

results_dir = ""
model = "knn"
if colab_env:
    # default to run in colab
    from google.colab import drive
    drive.mount('/content/drive')

    sys.path.insert(1, "/content/drive/Shareddrives/tcc_pos/utils")
    import model_utils

    results_dir = "/content/drive/Shareddrives/tcc_pos/results/"
    dataframe_path = "/content/drive/Shareddrives/tcc_pos/datasets/no_filtered_analysis.csv"   
else:
    # default to run local
    dataframe_path = "../../pre_processing/datasets/preprocessed_data/no_filtered_analysis.csv"

    sys.path.insert(1, "../utils/")
    import model_utils

    results_dir = "your_path"

In [ ]:
df = pd.read_csv(dataframe_path, sep = ",")

In [ ]:
# Drop RA column if exists
if "RA" in df.columns:
    print("Removing column RA")
    df.drop('RA', inplace=True, axis=1)

In [ ]:
df.head()

In [ ]:
# Run all predictions based on fold n_splits
def run_prediction(X_train, X_test, y_train, y_test):
    
    # used to evaluate best model params
    gridsearch_metrics = {
            'REC':'recall', 
            'AUC':'roc_auc', 
            'Accuracy': 'accuracy'
    }

    # model gridsearch params
    model_params = {
        'weights': ['uniform', 'distance'], 
        'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
        'p': [1, 2], # 1 = minkowski and 2 = euclidian
        'algorithm': ["auto"],
        "leaf_size": [10, 20, 30, 40, 50]
    }
    
    # gridsearch initialization
    clf = GridSearchCV(KNeighborsClassifier(),
                       n_jobs= 10,
                       cv = 2,
                       verbose= 1,
                       param_grid = model_params,
                       #scoring = gridsearch_metrics,
                       refit='accuracy')
    
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)

    predict_data = {
        "confusion_matrix": confusion_matrix(y_test, predictions),
        "classification_report": classification_report(y_test, predictions),
        "model_accuracy": accuracy_score(y_test, predictions),
        "f1_score": f1_score(y_test, predictions),
        "precision_score": precision_score(y_test, predictions),
        "roc_score": roc_auc_score(y_test, predictions),
        "recall_score": recall_score(y_test, predictions),
        "epoch_params": clf.best_params_,
        "max_error": max_error(y_test, predictions),
        "mean_squared_error": mean_squared_error(y_test, predictions),
        "mean_absolute_error": mean_absolute_error(y_test, predictions)
    }

    return predict_data

In [ ]:
def main():
    X = df.iloc[:, df.columns != "EVADIDO"].values
    y = df["EVADIDO"]

    skf = StratifiedKFold(n_splits = 10)
    skf.get_n_splits(X, y)
    scaler = StandardScaler()
    print(f"StratifiedKFold config: {skf} \n")
    all_predictions = []
    for train_index, test_index in skf.split(X, y):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # return best k element
        all_predictions.append(run_prediction(X_train, X_test, y_train, y_test))
        
    # General model outputs
    model_utils.generate_output(all_predictions)

    # General model outputs .csv
    model_utils.generate_output_csv(all_predictions)

    print("############################ FINISHED ############################")

In [ ]:
if __name__ == "__main__":
    main()